# Дипломный проект: Модель прогнозирования стоимости жилья для агентства недвижимости
## Машинное обучение

In [26]:
import pandas as pd
import numpy as np

In [43]:
data2 = pd.read_csv('data/ml/data2.csv')
target = pd.read_csv('data/ml/target.csv')

data2.set_index(data2.columns[0], inplace=True)
display(data2.head())

target = target.drop(target.columns[0], axis=1)
display(target.head())


,sqft,zipcode,PrivatePool,stories_num,age,mean_school_rating,schools_number,mean_school_distance,grades_pk,grades_k,...,street_type_way,property_type_apartment,property_type_co op,property_type_condo,property_type_land,property_type_manufactured home,property_type_multi family,property_type_other,property_type_single family,property_type_townhome
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1,7.000000,99216,0,2,4,4.00,3,1.33,1,0,...,0,0,0,0,0,0,0,0,1,0
2,30.502104,90049,1,1,62,6.67,3,1.96,0,1,...,0,0,0,0,0,0,0,0,1,0
3,7.000000,75205,0,3,17,9.25,4,0.75,1,0,...,0,0,0,0,0,0,0,0,1,0
9,1.930000,77080,0,2,4,3.00,3,1.07,1,0,...,0,0,0,0,0,0,0,0,1,0
12,9.000000,33028,1,1,27,7.33,3,2.04,1,0,...,0,0,0,0,0,0,0,0,1,0


,target
0,310000.0
1,2895000.0
2,2395000.0
3,311995.0
4,525000.0


## Машинное обучение: подбор оптимальной модели 

In [48]:
from sklearn.model_selection import train_test_split

# Разделение выборки на обучающую и тестовую
X_train, X_test, y_train, y_test = train_test_split(data2, target.values.reshape((len(target),)), test_size=0.3, random_state=42)


In [51]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Шкалирование данных
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Линейная и полиномиальная регрессия

In [30]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Создание и обучение модели
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Предсказание для обучающего набора данных
y_train_pred = model.predict(X_train_scaled)
r2_train = r2_score(y_train, y_train_pred)
print("R^2 для обучающего набора данных:", r2_train)

# Предсказание для тестового набора данных
y_test_pred = model.predict(X_test_scaled)
r2_test = r2_score(y_test, y_test_pred)
print("R^2 для тестового набора данных:", r2_test)

# Получили очень низкие значение - недообучение
# R^2 для обучающего набора данных: 0.20862092415078093
# R^2 для тестового набора данных: 0.21835417263592094

R^2 для обучающего набора данных: 0.20862092415078093
R^2 для тестового набора данных: 0.21835417263592094


In [52]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

# Отберем несколько (k) самых значимых признаков с помощью объекта SelectKBest
selector = SelectKBest(score_func=f_regression, k=25) 

# Преобразование данных с использованием SelectKBest на обучающей выборке
X_train_selected = selector.fit_transform(X_train, y_train)
print(X_train_selected.shape)

# Преобразование данных с использованием SelectKBest на тестовой выборке
X_test_selected = selector.transform(X_test)

(126340, 25)


In [53]:
# Попробуем полиномиальную регрессию 
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score

# Создание полиномиальных признаков
poly_features = PolynomialFeatures(degree=2)
# Будем использовать только важные признаки, полученные через SelectKBest
X_train_poly = poly_features.fit_transform(X_train_selected)
X_test_poly = poly_features.transform(X_test_selected)

# Создание и обучение модели полиномиальной регрессии
model = LinearRegression()
model.fit(X_train_poly, y_train)

# Предсказание для обучающего набора данных
y_train_pred = model.predict(X_train_poly)
r2_train = r2_score(y_train, y_train_pred)
print("R^2 для обучающего набора данных:", r2_train)

# Предсказание для тестового набора данных
y_test_pred = model.predict(X_test_poly)
r2_test = r2_score(y_test, y_test_pred)
print("R^2 для тестового набора данных:", r2_test)

# Снова низкий результат, причем есть признак переобучения
# R^2 для обучающего набора данных: 0.34705454030929583
# R^2 для тестового набора данных: 0.31259858893797554

R^2 для обучающего набора данных: 0.34705454030929583
R^2 для тестового набора данных: 0.31259858893797554


In [54]:
from sklearn.linear_model import Ridge

# Попробуем модель полиномиальной регрессии с регуляризацией
alpha = 0.1 
model = Ridge(alpha=alpha)
model.fit(X_train_poly, y_train)

# Предсказание для обучающего набора данных
y_train_pred = model.predict(X_train_poly)
r2_train = r2_score(y_train, y_train_pred)
print("R^2 для обучающего набора данных:", r2_train)

# Предсказание для тестового набора данных
y_test_pred = model.predict(X_test_poly)
r2_test = r2_score(y_test, y_test_pred)
print("R^2 для тестового набора данных:", r2_test)

# Переобучения нет, но результат низкий. Поищем другие модели
# R^2 для обучающего набора данных: 0.3470509728054677
# R^2 для тестового набора данных: 0.35181643386461203

R^2 для обучающего набора данных: 0.3470509728054677
R^2 для тестового набора данных: 0.35181643386461203


### Лес решений

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

model_rf = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=100)
model_rf.fit(X_train, y_train)

# Предсказание на тестовой выборке
y_pred = model_rf.predict(X_test)

# Расчет метрик для обучающей и тестовой выборок
train_r2 = r2_score(y_train, model_rf.predict(X_train))
test_r2 = r2_score(y_test, y_pred)

print("r2 на обучающей выборке:", train_r2)
print("r2 на тестовой выборке:", test_r2)

# Снова низкий результат
# r2 на обучающей выборке: 0.3097438800279121
# r2 на тестовой выборке: 0.3151616787157414

r2 на обучающей выборке: 0.3097438800279121
r2 на тестовой выборке: 0.3151616787157414


In [56]:
# Получение важности признаков
feature_importance = model_rf.feature_importances_
# Сортировка признаков по их важности
sorted_indices = feature_importance.argsort()[::-1]
# Получение имен признаков в порядке убывания важности
selected_features = data2.columns[sorted_indices]

print('Топ-20 важных признаков:\n', selected_features[:20])
# Сделаем обучающую и тестовую подвыборки из важных признаков
X_train_fi = X_train[selected_features[:20]]
X_test_fi = X_test[selected_features[:20]]

Топ-20 важных признаков:
 Index(['beds_num_others', 'CA', 'PrivatePool', 'zipcode', 'baths_num_others',
       'beds_num_5.0', 'stories_num', 'mean_school_rating', 'NY',
       'mean_school_distance', 'beds_num_2.0', 'beds_num_3.0', 'sqft',
       'baths_num_2.0', 'age', 'city_8', 'city_6', 'grades_k', 'city_4',
       'city_7'],
      dtype='object')


In [57]:
# Попробуем модель случайного леса на подвыборке 20 важных признаков
model_rf_fi = RandomForestRegressor(n_estimators=100, max_depth=5, min_samples_leaf=100)
model_rf_fi.fit(X_train_fi, y_train)

# Расчет метрик для обучающей и тестовой выборок
train_r2 = r2_score(y_train, model_rf_fi.predict(X_train_fi))
test_r2 = r2_score(y_test, model_rf_fi.predict(X_test_fi))

print("r2 на обучающей выборке:", train_r2)
print("r2 на тестовой выборке:", test_r2)

# Получили те же результаты, но быстрее
# r2 на обучающей выборке: 0.30768028721556606
# r2 на тестовой выборке: 0.314368903535059

r2 на обучающей выборке: 0.30768028721556606
r2 на тестовой выборке: 0.314368903535059


In [58]:
from sklearn.model_selection import RandomizedSearchCV

# Определение набора параметров для поиска
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'min_samples_split': [50, 100]
}

# Создание модели градиентного бустинга
model_rf_grid = RandomForestRegressor()

# Инициализация RandomizedSearchCV
random_search = RandomizedSearchCV(model_rf_grid, param_grid, n_iter=10)
# cv=5)

# Запуск поиска оптимальных параметров по выборке из 20 важных признаков
random_search.fit(X_train_fi, y_train)

# Вывод оптимальных параметров
print("Оптимальные параметры:", random_search.best_params_)

# Оптимальные параметры: {'n_estimators': 50, 'min_samples_split': 50, 'max_depth': 5} - 8min 30sec.

c:\Users\Питерские котики\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Оптимальные параметры: {'n_estimators': 50, 'min_samples_split': 50, 'max_depth': 5}


In [59]:
model_rf_fi = RandomForestRegressor(n_estimators=50, max_depth=5, min_samples_leaf=50)
model_rf_fi.fit(X_train_fi, y_train)

# Расчет метрик для обучающей и тестовой выборок
train_r2 = r2_score(y_train, model_rf_fi.predict(X_train_fi))
test_r2 = r2_score(y_test, model_rf_fi.predict(X_test_fi))

print("r2 на обучающей выборке:", train_r2)
print("r2 на тестовой выборке:", test_r2)

# r2 на обучающей выборке: 0.37483953902491607
# r2 на тестовой выборке: 0.3823269605487024
# Получили результат чуть выше

r2 на обучающей выборке: 0.37483953902491607
r2 на тестовой выборке: 0.3823269605487024


In [60]:
# Попробуем, вместо отсеивания признаков, использовать подвыборку строк
# Увеличим набор параметров для поиска
param_grid = {
    'n_estimators': [30, 50, 100],
    'max_depth': [3, 5, 8],
    'min_samples_split': [20, 50, 100]
}

# Создание модели градиентного бустинга
model_rf_grid = RandomForestRegressor()

# Инициализация RandomizedSearchCV
random_search = RandomizedSearchCV(model_rf_grid, param_grid, n_iter=10)
# cv=5)

# Запуск поиска оптимальных параметров
random_search.fit(X_train[:10000], y_train[:10000])

# Вывод оптимальных параметров
print("Оптимальные параметры:", random_search.best_params_)

# Оптимальные параметры: {'n_estimators': 50, 'min_samples_split': 20, 'max_depth': 8} - 1min  43sec

Оптимальные параметры: {'n_estimators': 50, 'min_samples_split': 20, 'max_depth': 8}


In [64]:
model_rf_row_slice = RandomForestRegressor(**random_search.best_params_)
model_rf_row_slice.fit(X_train, y_train)

# Расчет метрик для обучающей и тестовой выборок
train_r2 = r2_score(y_train, model_rf_row_slice.predict(X_train))
test_r2 = r2_score(y_test, model_rf_row_slice.predict(X_test))

print("r2 на обучающей выборке:", train_r2)
print("r2 на тестовой выборке:", test_r2)

# r2 на обучающей выборке: 0.6239188120512653
# r2 на тестовой выборке: 0.5080652367181127

r2 на обучающей выборке: 0.6239188120512653
r2 на тестовой выборке: 0.5080652367181127


### Градиентный бустинг

In [65]:

from sklearn.ensemble import GradientBoostingRegressor

# Создание и обучение модели градиентного бустинга
model_gb = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1, max_depth=8, min_samples_split=20)
model_gb.fit(X_train, y_train)

# Предсказание на обучающей и тестовой выборке
y_train_pred = model_gb.predict(X_train)
y_test_pred = model_gb.predict(X_test)

# Вычисление метрики R2 на обучающей и тестовой выборке
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R2 на обучающей выборке:", r2_train)
print("R2 на тестовой выборке:", r2_test)

# R2 на обучающей выборке: 0.843288611434015
# R2 на тестовой выборке: 0.5716147208646325
# Результат лучше, но переобучение

R2 на обучающей выборке: 0.843288611434015
R2 на тестовой выборке: 0.5716147208646325


In [66]:
# Определение набора параметров для поиска
param_grid = {
    'learning_rate': [0.1, 0.01],
    'n_estimators': [30, 50, 100],
    'max_depth': [3, 5, 8],
    'min_samples_split': [20, 50, 100]
}

# Создание модели градиентного бустинга
model_gb_grid = GradientBoostingRegressor()

# Инициализация RandomizedSearchCV
random_search = RandomizedSearchCV(model_gb_grid, param_grid, n_iter=10)
# cv=5)

# Запуск поиска оптимальных параметров
# Снова будем использовать часть строк, чтобы сократить время работы алгоритма
random_search.fit(X_train[:10000], y_train[:10000])

# Вывод оптимальных параметров
print("Оптимальные параметры:", random_search.best_params_)
# Оптимальные параметры: {'n_estimators': 100, 'min_samples_split': 20, 'max_depth': 5, 'learning_rate': 0.1} - 4 min


Оптимальные параметры: {'n_estimators': 50, 'min_samples_split': 20, 'max_depth': 5, 'learning_rate': 0.1}


In [ ]:
# Создание и обучение модели градиентного бустинга
model_gb_opt = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, min_samples_split=20)
model_gb_opt.fit(X_train, y_train)

# Предсказание на обучающей и тестовой выборке
y_train_pred = model_gb_opt.predict(X_train)
y_test_pred = model_gb_opt.predict(X_test)

# Вычисление метрики R2 на обучающей и тестовой выборке
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R2 на обучающей выборке:", r2_train)
print("R2 на тестовой выборке:", r2_test)

R2 на обучающей выборке: 0.6903700961853294
R2 на тестовой выборке: 0.5799659436741886


In [ ]:
# Остановимся на самой последней модели, сохраним ее в файл
import pickle

# Сохранение модели в файл
with open('model.pkl', 'wb') as file:
    pickle.dump(model_gb_opt, file)